# Bike Sharing Analytics

## Task

- Regression: Prediction of bike rental count hourly or daily based on the environmental and seasonal settings.

- Event and Anomaly Detection: Count of rented bikes are also correlated to some events in the town which easily are traceable via search engines. For instance, query like "2012-10-30 washington d.c." in Google returns related results to Hurricane Sandy. Some of the important events are identified in [1]. Therefore the data can be used for validation of anomaly or event detection algorithms as well.

[1] Fanaee-T, Hadi, and Gama, Joao, "Event labeling combining ensemble detectors and background knowledge", Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg, doi:10.1007/s13748-013-0040-3.


## Library

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import time
warnings.filterwarnings("ignore") 

## Data Wrangling

### Gathering Data

#### Daily data

In [51]:
# Load daily data
daily_df = pd.read_csv('./data/day.csv')
daily_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [52]:
daily_df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

The dataset contains various columns, including:  
- **instant**: Record index.  
- **dteday**: Date of the record.  
- **season**: The season in which the data was recorded (1: Spring, 2: Summer, 3: Fall, 4: Winter).  
- **yr**: Year of the data (0: 2011, 1: 2012).  
- **mnth**: Month of the year (1 to 12).  
- **holiday**: Whether the day is a holiday (1: Yes, 0: No).  
- **weekday**: Day of the week (0: Sunday, 6: Saturday).  
- **workingday**: Whether the day is a working day (1: Yes, 0: No).  
- **weathersit**: Weather situation (1: Clear, 2: Mist, 3: Light rain/snow, 4: Heavy rain/snow).  
- **temp** : Normalized temperature in Celsius. The values are divided to 41 (max).
- **atemp**: Normalized feeling temperature in Celsius. The values are divided to 50 (max).
- **hum**: Normalized humidity. The values are divided to 100 (max).
- **windspeed**: Normalized wind speed. The values are divided to 67 (max).
- **casual**: Count of casual users.
- **registered**: Count of registered users.
- **cnt**: Count of total rental bikes including both casual and registered.

By examining the first few rows, we can get an overview of the dataset before performing further analysis.  

#### Hourly data

In [53]:
# Load hour data
hourly_df = pd.read_csv('./data/hour.csv')
hourly_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


 This dataset contains similar features to the daily dataset (`day_df`), but with hourly records instead of daily aggregates. Some key columns include:  
- **instant**: Unique identifier for each row.  
- **dteday**: Date of the record.  
- **hr**: Hour of the day (0 to 23).  
- **season, yr, mnth, holiday, weekday, workingday, weathersit**: Similar to the daily dataset, representing seasonal and time-related information.  
- **temp, atemp, hum, windspeed**: Weather-related features.  
- **casual, registered, cnt**: Number of casual users, registered users, and total bike rentals for each hour.  

Since this dataset provides a more granular view with hourly records, it allows for a deeper analysis of bike-sharing trends throughout the day.  

### Assessing Data

#### Daily data

In [54]:
# Day data information
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB



- The dataset contains **731 rows** and **16 columns**.  
- All columns have **731 non-null values**, meaning there are **no missing values**.  
- The dataset consists of different data types:  
  - **Integer (`int64`)**: Used for categorical and count-based features such as `season`, `yr`, `mnth`, `holiday`, `weekday`, `workingday`, `weathersit`, `casual`, `registered`, and `cnt`.  
  - **Float (`float64`)**: Used for continuous numerical values like `temp`, `atemp`, `hum`, and `windspeed`.  
  - **Object (`object`)**: The `dteday` column, which represents dates in string format.  

This information helps in preprocessing and deciding if any columns need transformation (e.g., converting `dteday` to a proper datetime format).  

In [55]:
# Check Duplicate the day data
daily_df.duplicated().sum()

0

The output is **0**, which means there are no duplicate records in the dataset. This confirms that each row represents a unique entry, and no additional cleaning is required for duplicates.  

In [56]:
# Check Missing Value the day data
daily_df.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

The output shows that **all columns have 0 missing values**, meaning the dataset is complete and does not require imputation or removal of missing data.  

Since there are no missing values, we can proceed with the analysis without additional preprocessing for missing data.  

In [57]:
# The descriptive statistic from day data
daily_df.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


To understand the overall distribution of numerical variables, we use the `.describe()` function. This provides key statistical summaries, including:  
- **count**: Number of non-null values in each column.  
- **mean**: Average value of each column.  
- **std (standard deviation)**: Measures data dispersion from the mean.  
- **min and max**: Minimum and maximum values.  
- **25%, 50% (median), and 75%**: Quartiles, showing data distribution.  

Key Observations:  
- The dataset consists of **731 records**.  
- The **average number of total bike rentals (`cnt`) is 4504**, with a **minimum of 22** and a **maximum of 8714**.  
- The **temperature (`temp`) ranges from 0.06 to 0.86** (normalized values).  
- The dataset is **balanced across years (`yr`)**, with values of `0` (2011) and `1` (2012).  
- The **working day (`workingday`) and holiday (`holiday`) columns are binary**, indicating whether a day is a working day or holiday.  

This summary helps in understanding the overall trends and potential data distributions before further analysis.  

#### Hourly data

In [58]:
# Hour data information
hourly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


To understand the structure of the hourly dataset, we use the `.info()` function. This provides an overview of the dataset, including:  
- **Total entries**: 17,379 rows.  
- **Total columns**: 17 attributes.  
- **Data types**:  
  - `int64`: 12 columns (e.g., `season`, `yr`, `mnth`, `hr`, `holiday`, etc.).  
  - `float64`: 4 columns (`temp`, `atemp`, `hum`, `windspeed`).  
  - `object`: 1 column (`dteday`, representing dates).  
- **Memory usage**: 2.3+ MB.  

Key Observations:  
- The dataset contains **hourly records of bike rentals** over two years.  
- There are **no missing values** in any column, which simplifies data preprocessing.  
- The `hr` column represents hours of the day (0-23), allowing for time-based analysis.  
- `cnt` is the total number of rentals, obtained from the sum of `casual` and `registered` users.  

This summary helps in understanding the dataset before performing further exploratory data analysis.  


In [59]:
# Check duplicate the hour data
hourly_df.duplicated().sum()

0

The output is **0**, meaning there are **no duplicate entries** in the dataset. This confirms that each row represents a unique hourly record without redundancy.  

In [60]:
# Check Missing Value the hour data
hourly_df.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

The output shows **0 missing values** in all 17 columns. This means the dataset is **fully complete**, with no need for imputation or data cleaning related to missing values.  


In [61]:
# The descriptive statistic from day data
hourly_df.describe().round(2)

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.00,17379.00,17379.0,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00,17379.00
mean,8690.00,2.50,0.5,6.54,11.55,0.03,3.00,0.68,1.43,0.50,0.48,0.63,0.19,35.68,153.79,189.46
std,5017.03,1.11,0.5,3.44,6.91,0.17,2.01,0.47,0.64,0.19,0.17,0.19,0.12,49.31,151.36,181.39
min,1.00,1.00,0.0,1.00,0.00,0.00,0.00,0.00,1.00,0.02,0.00,0.00,0.00,0.00,0.00,1.00
25%,4345.50,2.00,0.0,4.00,6.00,0.00,1.00,0.00,1.00,0.34,0.33,0.48,0.10,4.00,34.00,40.00
50%,8690.00,3.00,1.0,7.00,12.00,0.00,3.00,1.00,1.00,0.50,0.48,0.63,0.19,17.00,115.00,142.00
75%,13034.50,3.00,1.0,10.00,18.00,0.00,5.00,1.00,2.00,0.66,0.62,0.78,0.25,48.00,220.00,281.00
max,17379.00,4.00,1.0,12.00,23.00,1.00,6.00,1.00,4.00,1.00,1.00,1.00,0.85,367.00,886.00,977.00


To understand the distribution of numerical features, we use the `.describe().round(2)` function. This provides key statistical summaries, including count, mean, standard deviation, min, max, and quartiles.  

Key Observations:  
- **Total Entries**: 17,379 rows.  
- **Hourly Distribution** (`hr`): The values range from **0 to 23**, covering all hours in a day.  
- **Temperature (`temp`) & Feels-Like Temperature (`atemp`)**:  
  - `temp` (Normalized between 0 and 1) has a **mean of 0.50**, suggesting an even spread over different weather conditions.  
  - `atemp` follows a similar distribution with a mean of **0.48**.  
- **Humidity (`hum`) & Wind Speed (`windspeed`)**:  
  - Humidity ranges from **0 to 1**, with a mean of **0.63**.  
  - Wind speed has an average of **0.19**, indicating most days experience mild winds.  
- **Bike Rentals (`cnt`)**:  
  - The total count (`cnt`) ranges from **1 to 977** per hour.  
  - Median rentals per hour is **142**, while the upper quartile reaches **281**, indicating some peak hours with high demand.  
  - `casual` riders (mean **35.68**) are significantly fewer than `registered` riders (mean **153.79**), suggesting that most users are registered members.  

Conclusion:  
This statistical summary provides insight into the distribution and variability of bike rentals across different hours, weather conditions, and user types. These insights will guide further analysis and model building.  

### Cleaning Data

#### Hourly data

In [62]:
hourly_df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [63]:
# Change the column names
hourly_column = {
    'instant' : 'index',
    'dteday': 'date',
    'yr': 'year',
    'mnth': 'month',
    'hr': 'hour',
    'temp': 'temperature',
    'atemp': 'feels_like_temperature',
    'hum': 'humidity',
    'cnt': 'count'
} 

hourly_df.rename(columns=hourly_column, inplace=True)

In [64]:
# Fix the datetime data
hourly_df['date'] = pd.to_datetime(hourly_df['date'])

In [65]:
# Categorical season column
season = {
    1 : 'springer',
    2 : 'summer',
    3 : 'fall',
    4 : 'winter'
}

hourly_df['season'] = hourly_df['season'].replace(season)

In [66]:
# Year column
year = {
    0: 2011,
    1: 2012
}

hourly_df['year'] = hourly_df['year'].replace(year)

In [67]:
# Month column
month = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

hourly_df['month'] = hourly_df['month'].replace(month)

In [68]:
# Hour Column
hourly_df['hour'] = hourly_df['hour'].apply(lambda x: time(x, 0))


In [69]:
# Change holiday column to categorical values
holiday = {
    0: 'No',
    1: 'Yes'
}

hourly_df['holiday'] = hourly_df['holiday'].replace(holiday)

In [70]:
# Change coloumns weekdays from numerical to categorical
weekdays = {
    0: 'Sunday',
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday'
}

hourly_df['weekday'] = hourly_df['weekday'].replace(weekdays)

In [71]:
# Change coloumns workingday from numerical to categorical
workingday = {
    0: 'No',
    1: 'Yes'
}

hourly_df['workingday'] = hourly_df['workingday'].replace(workingday)

In [72]:
# Change the weathersit from numerical to categorical
weathersit_labels = {
    1: 'Clear',
    2: 'Cloudy or Misty',
    3: 'Light Rain/Snow',
    4: 'Heavy Rain/Snow'
}

hourly_df['weathersit'] = hourly_df['weathersit'].map(weathersit_labels)

In [73]:
# Denormalized temperature column with multiple by 41
hourly_df['temperature'] = hourly_df['temperature'] * 41

In [74]:
# Denormalized feels_like_temperature column with multiple by 50
hourly_df['feels_like_temperature'] = hourly_df['feels_like_temperature'] * 50

In [75]:
# Denormalized humidity column with multiple by 100
hourly_df['humidity'] = hourly_df['humidity'] * 100

In [76]:
# Denormalized windspeed column with multiple by 67
hourly_df['windspeed'] = hourly_df['windspeed'] * 67 

In [77]:
hourly_df.head()

,index,date,season,year,month,hour,holiday,weekday,workingday,weathersit,temperature,feels_like_temperature,humidity,windspeed,casual,registered,count
0,1,2011-01-01,springer,2011,January,00:00:00,No,Saturday,No,Clear,9.84,14.395,81.0,0.0,3,13,16
1,2,2011-01-01,springer,2011,January,01:00:00,No,Saturday,No,Clear,9.02,13.635,80.0,0.0,8,32,40
2,3,2011-01-01,springer,2011,January,02:00:00,No,Saturday,No,Clear,9.02,13.635,80.0,0.0,5,27,32
3,4,2011-01-01,springer,2011,January,03:00:00,No,Saturday,No,Clear,9.84,14.395,75.0,0.0,3,10,13
4,5,2011-01-01,springer,2011,January,04:00:00,No,Saturday,No,Clear,9.84,14.395,75.0,0.0,0,1,1


#### Daily data

In [78]:
daily_df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [79]:
# Change the column names
daily_columns = {
    'instant' : 'index',
    'dteday': 'date',
    'yr': 'year',
    'mnth': 'month',
    'temp': 'temperature',
    'atemp': 'feels_like_temperature',
    'hum': 'humidity',
    'cnt': 'count'
} 

daily_df.rename(columns=daily_columns, inplace=True)

In [80]:
# Fix the datetime data
daily_df['date'] = pd.to_datetime(daily_df['date'])

In [81]:
# Categorical season column
season = {
    1 : 'springer',
    2 : 'summer',
    3 : 'fall',
    4 : 'winter'
}

daily_df['season'] = daily_df['season'].replace(season)

In [82]:
# Year column
year = {
    0: 2011,
    1: 2012
}

daily_df['year'] = daily_df['year'].replace(year)

In [83]:
# Month column
month = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

daily_df['month'] = daily_df['month'].replace(month)

In [84]:
# Change holiday column to categorical values
holiday = {
    0: 'No',
    1: 'Yes'
}

daily_df['holiday'] = daily_df['holiday'].replace(holiday)

In [85]:
# Change coloumns weekdays from numerical to categorical
weekdays = {
    0: 'Sunday',
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday'
}

daily_df['weekday'] = daily_df['weekday'].replace(weekdays)

In [86]:
# Change coloumns workingday from numerical to categorical
workingday = {
    0: 'No',
    1: 'Yes'
}

daily_df['workingday'] = daily_df['workingday'].replace(workingday)

In [87]:
# Change the weathersit from numerical to categorical
weathersit_labels = {
    1: 'Clear',
    2: 'Cloudy or Misty',
    3: 'Light Rain/Snow',
    4: 'Heavy Rain/Snow'
}

daily_df['weathersit'] = daily_df['weathersit'].map(weathersit_labels)

In [88]:
# Denormalized temperature column with multiple by 41
daily_df['temperature'] = daily_df['temperature'] * 41

In [89]:
# Denormalized feels_like_temperature column with multiple by 50
daily_df['feels_like_temperature'] = daily_df['feels_like_temperature'] * 50

In [90]:
# Denormalized humidity column with multiple by 100
daily_df['humidity'] = daily_df['humidity'] * 100

In [91]:
# Denormalized windspeed column with multiple by 67
daily_df['windspeed'] = daily_df['windspeed'] * 67 

In [92]:
daily_df.head()

,index,date,season,year,month,holiday,weekday,workingday,weathersit,temperature,feels_like_temperature,humidity,windspeed,casual,registered,count
0,1,2011-01-01,springer,2011,January,No,Saturday,No,Cloudy or Misty,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,2011-01-02,springer,2011,January,No,Sunday,No,Cloudy or Misty,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,2011-01-03,springer,2011,January,No,Monday,Yes,Clear,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,2011-01-04,springer,2011,January,No,Tuesday,Yes,Clear,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,2011-01-05,springer,2011,January,No,Wednesday,Yes,Clear,9.305237,11.46350,43.6957,12.522300,82,1518,1600


## Exploratory Data Analysis (EDA)

## Visualization & Explanatory Analysis

### Question 1: What are the most and least frequently rented hours?

In [ ]:
sum_order = hour_df.groupby("hours").count_cr.sum().sort_values(ascending=False).reset_index()

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(30, 15))
sns.barplot(
    x="hours", 
    y="count_cr", 
    data=sum_order.head(5), 
    ax=ax[0],
    color='#ff9999'
)

ax[0].set_ylabel(None)
ax[0].set_xlabel("Hours (PM)", fontsize=30)
ax[0].set_title("Hours with lots of bike renters", loc="center", fontsize=30)
ax[0].tick_params(axis='y', labelsize=35)
ax[0].tick_params(axis='x', labelsize=30)

sns.barplot(
    x="hours", 
    y="count_cr", 
    data=sum_order.sort_values(
        by="hours", 
        ascending=True
    ).head(5), 
    ax=ax[1],
    color='#ff9999'
)

ax[1].set_ylabel(None)
ax[1].set_xlabel("Hours (AM)",  fontsize=30)
ax[1].set_title("Hours with few bike renters", loc="center", fontsize=30)
ax[1].invert_xaxis()
ax[1].yaxis.set_label_position("right")
ax[1].yaxis.tick_right()
ax[1].tick_params(axis='y', labelsize=35)
ax[1].tick_params(axis='x', labelsize=30)

plt.show()


### Question 2: In what season is bike rental peak?

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

sns.barplot(
    y="count_cr",
    x="season",
    data= day_df.sort_values(
        by="season",
        ascending=False
    ),
    ax=ax,
    errorbar=None,
    color='#ff9999'
)

ax.set_title("Inter-seasonal Chart", loc="center", fontsize=16)
ax.set_ylabel(None)
ax.set_xlabel(None)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

plt.show()

### Question 3: How has the company's sales performance been in recent years?

In [ ]:
plt.figure(figsize=(25,5))

monthly_ct = day_df['count_cr'].groupby(day_df['dteday']).max()

plt.scatter(
    monthly_ct.index,
    monthly_ct.values,
    marker='o',
    color='#ff9999',
    s=10
)
plt.plot(monthly_ct.index,monthly_ct.values)
plt.xlabel('Bulan')
plt.ylabel('Number')
plt.title('Number of Customers per Month in 2012')

plt.show()

### Question 4: How do registered customers compare to casual customers?

In [ ]:
casual_total = sum(day_df['casual'])
total_regiter = sum(day_df['registered'])
data = [total_regiter, casual_total]
labels = ["Registered", "Casual"]
myexplode = [0, 0.1]
colors = [ '#66b3ff', '#ff9999']

plt.pie(data, labels=labels, autopct='%1.1f%%', colors=colors, explode=myexplode)
plt.show()

## Conclusion

- **Question 1:** What are the most and least frequently rented hours?
- **Question 2:** In what season is bike rental peak?
- **Question 3:** How has the company's sales performance been in recent years?
- **Question 4:** How do registered customers compare to casual customers?

**Conclution pertanyaan 1:** Based on the image above, you can see that bike rentals are most used at 17:00. In contrast, rentals at 04:00 are the least sold products.

**conclution pertanyaan 2:** in the Fall season.

**conclution pertanyaan 3:** 
Based on the visualization above, we can see that the highest number of orders occured in September 2012,
we can also see significant decrease in the number of orders in January 2011

**conclution pertanyaan 4:** Those who have registered 81.2%, while those who have not registered (casual) are 18.8% 